# Lecture 6 — Introduction to Statistical Learning

In this lecture, we will walk through a complete data science research pipeline using scikit-learn. This lecture aims to show you the basic steps of data science with two distinct objectives: regression and binary classification. We will only cover basic linear models in this lecture but the same pipeline can be applied to more complex models that we will cover next week.

In [ ]:
# install scikit-learn if you haven't done so.
# ! pip install scikit-learn

In [ ]:
from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt

import sklearn

In [ ]:
# Setup the computational environment.
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# fix the random seed for reproducibility.
RANDOM_STATE = 42

# Create outputs dir
Path("../outputs").mkdir(parents=True, exist_ok=True)
print("Setup complete.")

## Notations and High-Level Picture

### Input Data
- Tabular features: $X \in \mathbb{R}^{n \times p}$
  - $n$: number of samples (rows)
  - $p$: number of features (columns)

### Target/Objective
  - Regression: $y \in \mathbb{R}^{n}$
  - Classification (binary): $y \in \{0,1\}^{n}$
  - Build a model $f: X \to y$ to predict the target $\hat{y}$ from the features $X$.
  - The model $f$ is a function that maps the features $X$ to the target $y$.
  - We want to find the best model $f$ that minimizes the prediction error (i.e., the difference between $\hat{y}$ and $y$).

### Split
  - Randomly split the data into training and test sets: $(X_{\text{train}}, y_{\text{train}}), (X_{\text{test}}, y_{\text{test}})$
  - Train a model using the training set and evaluate on the test set.

## Retrieve the Data for Binary Classification: [Breast Cancer Wisconsin (Diagnostic) Dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset)

**Data Set Characteristics:**

- **Number of Instances:** 569
- **Number of Attributes:** 30 numeric, predictive attributes and the class

**Attribute Information:**

- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter² / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry
- fractal dimension ("coastline approximation" - 1)

The mean, standard error, and "worst" or largest (mean of the three worst/largest values) of these features were computed for each image, resulting in 30 features. For instance, field 0 is Mean Radius, field 10 is Radius SE, field 20 is Worst Radius.

**Classes:**

- WDBC-Malignant
- WDBC-Benign

In [ ]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)

In [ ]:
# X here is a pandas DataFrame.
X

In [ ]:
# y is a pandas Series.
y

In [ ]:
y.value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.ylabel('')
plt.title('Class Distribution')
plt.show()

# Data Transformation/Normalization/Standardization

In [ ]:
fig, axes = plt.subplots(6, 5, figsize=(15, 12))
X.hist(ax=axes, bins=20, edgecolor='black', alpha=0.7)
plt.tight_layout()
plt.show()

## 1. Normalization with MinMaxScaler
> Normalization rescales each feature to a **fixed range**, commonly $[0, 1]$ or $[-1, 1]$.

**Formula (min–max normalization):**
$$
x'_i = \frac{x_i - x_{\min}}{x_{\max} - x_{\min}}
$$

where  
- $x_{\min}$ and $x_{\max}$ are the minimum and maximum values of the feature.

**Why normalize:**
- Keeps features within a bounded range, useful for models sensitive to scale (e.g., neural networks, k-NN).  
- Improves convergence during gradient-based optimization.  
- Maintains interpretability when combining heterogeneous units or feature types.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data column by column
X_normalized = X.copy()

# create the min-max scaler
scaler = MinMaxScaler()

for col in X.columns:
    # use fit-transform to normalize the data
    X_normalized[col] = scaler.fit_transform(X[[col]])

In [ ]:
# Create side-by-side histograms
fig, axes = plt.subplots(6, 10, figsize=(20, 9), dpi=300)

# Original data on the left (first 5 columns)
for i, col in enumerate(X.columns):
    row = i // 5
    col_idx = i % 5
    X[col].hist(ax=axes[row, col_idx], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[row, col_idx].set_title(f'{col}\n(Original)', fontsize=8)
    axes[row, col_idx].tick_params(labelsize=6)

# Normalized data on the right (second 5 columns)
for i, col in enumerate(X_normalized.columns):
    row = i // 5
    col_idx = (i % 5) + 5
    X_normalized[col].hist(ax=axes[row, col_idx], bins=20, edgecolor='black', alpha=0.7, color='coral')
    axes[row, col_idx].set_title(f'{col}\n(Normalized)', fontsize=8)
    axes[row, col_idx].tick_params(labelsize=6)

plt.tight_layout()
plt.show()


## 2. Standardization
> Standardization rescales each feature so that it has **mean = 0** and **standard deviation = 1**.

**Formula:**
$$
z_i = \frac{x_i - \mu}{\sigma}
$$

where  
- $x_i$ is the original value,  
- $\mu$ is the mean of the feature,  
- $\sigma$ is the standard deviation.

**Why standardize:**
- Ensures all features contribute equally in distance- or gradient-based models (e.g., linear/logistic regression, SVM, PCA).  
- Prevents large-scale variables from dominating smaller-scale ones.  
- Less sensitive to outliers than normalization.
- Makes model training more stable and interpretable.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize the data column by column
X_standardized = X.copy()

# create the standard scaler
scaler = StandardScaler()

for col in X.columns:
    # use fit-transform to standardize the data
    X_standardized[col] = scaler.fit_transform(X[[col]])

# Create side-by-side histograms
fig, axes = plt.subplots(6, 10, figsize=(20, 9), dpi=300)

# Original data on the left (first 5 columns)
for i, col in enumerate(X.columns):
    row = i // 5
    col_idx = i % 5
    X[col].hist(ax=axes[row, col_idx], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[row, col_idx].set_title(f'{col}\n(Original)', fontsize=8)
    axes[row, col_idx].tick_params(labelsize=6)

# Standardized data on the right (second 5 columns)
for i, col in enumerate(X_standardized.columns):
    row = i // 5
    col_idx = (i % 5) + 5
    X_standardized[col].hist(ax=axes[row, col_idx], bins=20, edgecolor='black', alpha=0.7, color='mediumseagreen')
    axes[row, col_idx].set_title(f'{col}\n(Standardized)', fontsize=8)
    axes[row, col_idx].tick_params(labelsize=6)

plt.tight_layout()
plt.show()


## 3. Log Transformation
> Log transformation compresses large values and expands small ones, reducing skewness in right-tailed distributions.

**Formula:**
$$
x'_i = \log(x_i + c)
$$

where
- $x_i$ is the original value,
- $c$ is a constant (often $c = 1$) to handle zero or negative values.

**Why apply log transformation:**
- Reduces the impact of outliers and right-skewed distributions.
- Makes multiplicative relationships approximately additive.
- Stabilizes variance and improves linear model assumptions.
- Useful for highly skewed data such as income, citation counts, or population sizes.


In [ ]:
# Apply log transformation to the original data
X_log = X.copy()
for col in X_log.columns:
    X_log[col] = np.log(X_log[col] + 1)  # Add 1 to handle zeros

# Create side-by-side histograms
fig, axes = plt.subplots(6, 10, figsize=(20, 9), dpi=300)

# Original data on the left (first 5 columns)
for i, col in enumerate(X.columns):
    row = i // 5
    col_idx = i % 5
    X[col].hist(ax=axes[row, col_idx], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[row, col_idx].set_title(f'{col}\n(Original)', fontsize=8)
    axes[row, col_idx].tick_params(labelsize=6)

# Log-transformed data on the right (second 5 columns)
for i, col in enumerate(X_log.columns):
    row = i // 5
    col_idx = (i % 5) + 5
    X_log[col].hist(ax=axes[row, col_idx], bins=20, edgecolor='black', alpha=0.7, color='green')
    axes[row, col_idx].set_title(f'{col}\n(Log-transformed)', fontsize=8)
    axes[row, col_idx].tick_params(labelsize=6)

plt.tight_layout()
plt.show()

## 4. Summary of Data Transformation Techniques
| Transformation      | Typical Use Case                                   | Range / Scale     | Key Benefit                           |
|----------------------|----------------------------------------------------|-------------------|----------------------------------------|
| Standardization      | Features with different means/variances           | Mean 0, SD 1      | Equal weighting across features and less sensitive to outliers        |
| Normalization        | Scale-sensitive models (e.g., NN, k-NN)           | [0, 1] or [-1, 1] | Bounded magnitude, stable gradients    |
| Log Transformation   | Skewed, positive-valued data                      | Unbounded         | Reduces skewness, stabilizes variance  |

# Data Splitting with `sklearn.model_selection.train_test_split`
- Data splitting is essential to ensure that our model’s performance reflects genuine predictive ability rather than overfitting to a particular dataset.
- By dividing the data into training, validation, and test sets, we can train the model on one subset, tune hyperparameters on another, and finally evaluate it on unseen data.
- This separation allows for an unbiased assessment of generalization performance and helps prevent overly optimistic estimates that might arise if the same data were used for both fitting and evaluation.

In the data science pipeline, we want to split the data into three parts: the training set, the validation set, and the test set.

- Training set (typically 70% of the data) – The portion of the data used to fit the model. The model learns patterns, parameters, or relationships from this subset.
- Validation set (typically 15% of the data) – A separate subset used to tune hyperparameters and prevent overfitting. It helps select the best model configuration without touching the final test data.
- Test set (typically 15% of the data) – The held-out portion of data used only for final evaluation. It provides an unbiased estimate of generalization performance on unseen data after all model choices are finalized.

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.5, random_state=RANDOM_STATE
)

X_val, X_test, y_val, y_test = sklearn.model_selection.train_test_split(
    X_val_and_test, y_val_and_test, test_size=0.5, random_state=RANDOM_STATE
)

print(f"Training set: {X_train.shape}, {len(X_train)} samples, {y_train.sum()} positive ({y_train.mean():.2%})")
print(f"Validation set: {X_val.shape}, {len(X_val)} samples, {y_val.sum()} positive ({y_val.mean():.2%})")
print(f"Test set: {X_test.shape}, {len(X_test)} samples, {y_test.sum()} positive ({y_test.mean():.2%})")

# Visualize the class distribution in the three splits
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Training set pie chart
n_train_neg = len(y_train) - y_train.sum()
n_train_pos = y_train.sum()
axes[0].pie([n_train_neg, n_train_pos],
            labels=[f'Negative (n={n_train_neg})', f'Positive (n={n_train_pos})'],
            autopct='%1.1f%%',
            startangle=90,
            colors=['#ff9999', '#66b3ff'])
axes[0].set_title(f'Training Set\n(n={len(y_train)})')

# Validation set pie chart
n_val_neg = len(y_val) - y_val.sum()
n_val_pos = y_val.sum()
axes[1].pie([n_val_neg, n_val_pos],
            labels=[f'Negative (n={n_val_neg})', f'Positive (n={n_val_pos})'],
            autopct='%1.1f%%',
            startangle=90,
            colors=['#ff9999', '#66b3ff'])
axes[1].set_title(f'Validation Set\n(n={len(y_val)})')

# Test set pie chart
n_test_neg = len(y_test) - y_test.sum()
n_test_pos = y_test.sum()
axes[2].pie([n_test_neg, n_test_pos],
            labels=[f'Negative (n={n_test_neg})', f'Positive (n={n_test_pos})'],
            autopct='%1.1f%%',
            startangle=90,
            colors=['#ff9999', '#66b3ff'])
axes[2].set_title(f'Test Set\n(n={len(y_test)})')

plt.tight_layout()
plt.show()


#### Notes on `stratify`: `stratify` is used to ensure that the class distribution in the training, validation, and test sets is the same as in the original data.

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.5, stratify=y, random_state=RANDOM_STATE
)

X_val, X_test, y_val, y_test = sklearn.model_selection.train_test_split(
    X_val_and_test, y_val_and_test, test_size=0.5, stratify=y_val_and_test, random_state=RANDOM_STATE
)

print(f"Training set: {X_train.shape}, {len(X_train)} samples, {y_train.sum()} positive ({y_train.mean():.2%})")
print(f"Validation set: {X_val.shape}, {len(X_val)} samples, {y_val.sum()} positive ({y_val.mean():.2%})")
print(f"Test set: {X_test.shape}, {len(X_test)} samples, {y_test.sum()} positive ({y_test.mean():.2%})")

In [ ]:
# Visualize the class distribution in the three splits
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Training set pie chart
n_train_neg = len(y_train) - y_train.sum()
n_train_pos = y_train.sum()
axes[0].pie([n_train_neg, n_train_pos],
            labels=[f'Negative (n={n_train_neg})', f'Positive (n={n_train_pos})'],
            autopct='%1.1f%%',
            startangle=90,
            colors=['#ff9999', '#66b3ff'])
axes[0].set_title(f'Training Set\n(n={len(y_train)})')

# Validation set pie chart
n_val_neg = len(y_val) - y_val.sum()
n_val_pos = y_val.sum()
axes[1].pie([n_val_neg, n_val_pos],
            labels=[f'Negative (n={n_val_neg})', f'Positive (n={n_val_pos})'],
            autopct='%1.1f%%',
            startangle=90,
            colors=['#ff9999', '#66b3ff'])
axes[1].set_title(f'Validation Set\n(n={len(y_val)})')

# Test set pie chart
n_test_neg = len(y_test) - y_test.sum()
n_test_pos = y_test.sum()
axes[2].pie([n_test_neg, n_test_pos],
            labels=[f'Negative (n={n_test_neg})', f'Positive (n={n_test_pos})'],
            autopct='%1.1f%%',
            startangle=90,
            colors=['#ff9999', '#66b3ff'])
axes[2].set_title(f'Test Set\n(n={len(y_test)})')

plt.tight_layout()
plt.show()


## Apply the standardization transformation for the training, validation, and test sets.
- Note that we are fitting for the training set and then applying the same transformation to the validation and test sets.
- Avoid data leakage: Using validation/test data to compute mean/std “peeks” at their distribution, leaking information into preprocessing and inflating metrics.
- Unbiased evaluation: Metrics should reflect how the model performs on unseen data; scaling with only training stats keeps the test truly unseen.
- Deployment match: At inference you won’t have future data to recompute scaling; you apply the fixed scaler learned from historical (training) data.

In [ ]:
# apply the standardization transformation here for example.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# note that we are fitting for the training set and then applying the same transformation to the validation and test sets.
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Original training set mean: {X_train.mean().mean():.4f}, std: {X_train.std().mean():.4f}")
print(f"Scaled training set mean: {X_train_scaled.mean():.4f}, std: {X_train_scaled.std():.4f}")

print(f"Original validation set mean: {X_val.mean().mean():.4f}, std: {X_val.std().mean():.4f}")
print(f"Scaled validation set mean: {X_val_scaled.mean():.4f}, std: {X_val_scaled.std():.4f}")

print(f"Original test set mean: {X_test.mean().mean():.4f}, std: {X_test.std().mean():.4f}")
print(f"Scaled test set mean: {X_test_scaled.mean():.4f}, std: {X_test_scaled.std():.4f}")

# Estimate our First Model (Logistic Regression)
## 1. Create the `LogisticRegression` model object
- The logistic regression model is solved using an iterative optimization algorithm instead of using a closed-form solution.
- So we would need to specify the maximum number of iterations using `max_iter`. 
- We want to set a `max_iter` value sufficiently large (e.g., 1000 here, but depending on your actual dataset) so that the model parameters could converge within this window.
- There is an internal "early-stopping" design, the model estimation will stop early if it detects convergence.
- The `random_state` argument helps you fix the randomness in the algorithm (e.g., parameter initialization, data ordering).

In [ ]:
# Fit a logistic regression model
model = sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=RANDOM_STATE, verbose=2)
model

Here is a list of parameters that you can tune for the `LogisticRegression` model.
| Parameter | Description |
|------------|--------------|
| **penalty** | Specifies the norm used in the penalization. Options: `'l1'`, `'l2'`, `'elasticnet'`, or `'none'`. Default is `'l2'`. |
| **dual** | Whether to use the dual or primal formulation. `True` is only supported for `'liblinear'` solver and useful when `n_samples < n_features`. |
| **tol** | Tolerance for stopping criteria; smaller values lead to more precise convergence. |
| **C** | Inverse of regularization strength. Smaller values mean stronger regularization. |
| **fit_intercept** | If `True`, adds an intercept term (bias) to the decision function. |
| **intercept_scaling** | Used only when `solver='liblinear'` and `fit_intercept=True`; scales the synthetic feature added to the intercept. |
| **class_weight** | Assigns weights to classes to handle imbalance. Options: `None`, `'balanced'`, or a dictionary mapping classes to weights. |
| **random_state** | Controls random number generation for reproducibility (e.g., in solver initialization or data shuffling). |
| **solver** | Optimization algorithm to use: `'lbfgs'`, `'liblinear'`, `'saga'`, `'newton-cg'`, or `'sag'`. `'lbfgs'` works well for most problems. |
| **max_iter** | Maximum number of iterations taken for the solvers to converge. Increase if you get convergence warnings. |
| **multi_class** | Defines multiclass strategy: `'auto'`, `'ovr'` (one-vs-rest), or `'multinomial'`. `'auto'` chooses based on the solver. |
| **verbose** | Controls verbosity of solver output (integer). |
| **warm_start** | If `True`, reuse the previous solution to speed up convergence for similar data. |
| **n_jobs** | Number of CPU cores used during computation. `None` = 1 core, `-1` = all cores. |
| **l1_ratio** | Used only when `penalty='elasticnet'`; mixes L1 and L2 regularization (0 = L2 only, 1 = L1 only). |


## 2. Call the `fit()` method on the logistic regression to estimate model parameters.

In [ ]:
model.fit(X_train_scaled, y_train)

## 3.1 Make predictions on the validation set with `predict()`, generates the class labels.

In [ ]:
# Make predictions on the validation set
y_val_pred = model.predict(X_val_scaled)

In [ ]:
y_val_pred

## 3.2 Make predictions on the validation set with `predict_proba()`, generates the probability of the positive and negative classes. It will give you predicted probabilities for all classes if multi-class classification is used.

In [ ]:
model.predict_proba(X_val_scaled)

In [ ]:
# we are taking the second column of the output, which is the probability of the positive class (1).
y_val_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
y_val_pred_proba

In [ ]:
# Visualize predicted class labels vs predicted probabilities
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Histogram of predicted probabilities by predicted class
axes[0].hist(y_val_pred_proba[y_val_pred == 0], bins=30, alpha=0.6, label='Predicted Class 0', color='blue')
axes[0].hist(y_val_pred_proba[y_val_pred == 1], bins=30, alpha=0.6, label='Predicted Class 1', color='red')
axes[0].axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold (0.5)')
axes[0].set_xlabel('Predicted Probability')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Predicted Probabilities by Predicted Class')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Scatter plot of predicted probabilities colored by actual class
scatter_0 = axes[1].scatter(range(len(y_val_pred_proba[y_val == 0])),
                            y_val_pred_proba[y_val == 0],
                            alpha=0.6, label='Actual Class 0', color='blue', s=30)
scatter_1 = axes[1].scatter(range(len(y_val_pred_proba[y_val == 0]),
                                  len(y_val_pred_proba[y_val == 0]) + len(y_val_pred_proba[y_val == 1])),
                            y_val_pred_proba[y_val == 1],
                            alpha=0.6, label='Actual Class 1', color='red', s=30)
axes[1].axhline(y=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold (0.5)')
axes[1].set_xlabel('Sample Index')
axes[1].set_ylabel('Predicted Probability')
axes[1].set_title('Predicted Probabilities by Actual Class')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



In [ ]:
# we can do the same on the training set.
y_train_pred = model.predict(X_train_scaled)
y_train_pred_proba = model.predict_proba(X_train_scaled)[:, 1]

## 4. Evaluate the model

### **Accuracy**
Accuracy measures the proportion of correctly predicted instances among all instances.  
$$
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}} = \frac{TP + TN}{TP + TN + FP + FN}
$$

It is suitable when classes are balanced and misclassification costs are equal.

Examples:
```python
from sklearn.metrics import accuracy_score

# Basic usage
y_true = [0, 1, 1, 0]
y_pred = [0, 1, 0, 0]
print(accuracy_score(y_true, y_pred))  # 0.75

# With sample weights
weights = [1.0, 2.0, 1.0, 1.0]
print(accuracy_score(y_true, y_pred, sample_weight=weights))
```

**API reference:** [sklearn.metrics.accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)


In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy on validation set
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Calculate accuracy on training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")


---


### **Confusion Matrix**
A confusion matrix summarizes prediction results by comparing actual vs. predicted classes.

|                | **Predicted Positive** | **Predicted Negative** |
|----------------|------------------------|------------------------|
| **Actual Positive** | True Positive (TP)       | False Negative (FN)      |
| **Actual Negative** | False Positive (FP)      | True Negative (TN)       |

From the confusion matrix, we can derive:
- **Precision:** $ \displaystyle \frac{TP}{TP + FP} $
- **Recall (Sensitivity):** $ \displaystyle \frac{TP}{TP + FN} $
- **Specificity:** $ \displaystyle \frac{TN}{TN + FP} $
- **Accuracy:** $ \displaystyle \frac{TP + TN}{TP + TN + FP + FN} $

Examples:
```python
from sklearn.metrics import confusion_matrix

y_true = [0, 0, 1, 1]
y_pred = [0, 1, 0, 1]
print(confusion_matrix(y_true, y_pred))

# Explicit label order
print(confusion_matrix(y_true, y_pred, labels=[1, 0]))

# Normalized by true labels (rows)
print(confusion_matrix(y_true, y_pred, normalize='true'))
```

**API reference:** [sklearn.metrics.confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Compute confusion matrix for validation set
cm = confusion_matrix(y_val, y_val_pred)
print("Confusion Matrix (Validation Set):")
print(cm)

# Extract metrics from confusion matrix
tn = cm[0, 0]
fp = cm[0, 1]
fn = cm[1, 0]
tp = cm[1, 1]
print(f"\nTrue Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

# Calculate derived metrics
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)

print(f"\nPrecision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Accuracy: {accuracy:.4f}")



---

### **F1 Score**
The F1 score is the harmonic mean of precision and recall, balancing both metrics.  
It is particularly useful when dealing with imbalanced datasets.

$$
\text{F1} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} = \frac{2TP}{2TP + FP + FN}
$$

- **Range:** $0$ (worst) → $1$ (best)
- **Interpretation:** High F1 indicates both low false positives and false negatives.


Examples:
```python
from sklearn.metrics import f1_score

# Binary F1
y_true = [0, 0, 1, 1]
y_pred = [0, 1, 1, 1]
print(f1_score(y_true, y_pred))  # default 'binary'

# Multiclass F1 with averaging
y_true = [0, 1, 2, 2]
y_pred = [0, 2, 1, 2]
print(f1_score(y_true, y_pred, average='macro'))
print(f1_score(y_true, y_pred, average='weighted'))
```

**API reference:** [sklearn.metrics.f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
from sklearn.metrics import f1_score

# Calculate F1 score
f1 = f1_score(y_val, y_val_pred)
print(f"F1 Score: {f1:.4f}")

# Manual calculation to verify
f1_manual = 2 * (precision * recall) / (precision + recall)
print(f"F1 Score (manual): {f1_manual:.4f}")

## Probability ($\hat{p}$-Based) Metrics


---

### **Cross-Entropy Loss**
Cross-entropy loss (or log loss) quantifies the difference between the true labels $y$ and the predicted probabilities $\hat{y}$.  

For **binary classification**:
$$
L = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$

For **multi-class classification**:
$$
L = -\frac{1}{N} \sum_{i=1}^{N} \sum_{c=1}^{C} y_{i,c} \log(\hat{y}_{i,c})
$$

Examples:
```python
from sklearn.metrics import log_loss

# Binary: y_true (0/1) and predicted probabilities for class 1
y_true = [0, 1, 1, 0]
y_prob = [0.1, 0.9, 0.4, 0.2]
print(log_loss(y_true, y_prob))

# Multiclass: probability matrix with columns per class
y_true = [0, 2, 1]
y_proba = [[0.7, 0.2, 0.1],
           [0.1, 0.2, 0.7],
           [0.2, 0.7, 0.1]]
print(log_loss(y_true, y_proba, labels=[0, 1, 2]))
```

**API reference:** [sklearn.metrics.log_loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html)


In [ ]:
from sklearn.metrics import log_loss

# Calculate cross-entropy loss
ce_loss = log_loss(y_val, y_val_pred_proba)
print(f"Cross-Entropy Loss: {ce_loss:.4f}")


It penalizes confident wrong predictions heavily and is widely used in neural network training.

---

### **AUC (Area Under the ROC Curve)**
AUC measures the ability of a model to rank positive samples higher than negative ones.  
It corresponds to the area under the ROC curve (True Positive Rate vs. False Positive Rate).

- **Interpretation:** Probability that a randomly chosen positive instance is ranked above a randomly chosen negative instance.
- **Range:** $0.5$ (random guessing) → $1.0$ (perfect classifier)

**API references:**
- [sklearn.metrics.roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
- [sklearn.metrics.roc_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Calculate AUC
auc_score = roc_auc_score(y_val, y_val_pred_proba)
print(f"AUC Score: {auc_score:.4f}")

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_score:.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier (AUC = 0.5)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(alpha=0.3)
plt.show()



---

### **(Bonus) Calibration**
Calibration evaluates how well predicted probabilities align with actual outcome frequencies.  
A well-calibrated model’s output probabilities reflect true likelihoods (e.g., predictions of $\hat{p}=0.8$ are correct about 80% of the time).

Common tools:
- **Reliability Diagram:** Plots predicted probabilities vs. empirical accuracies.
- **Expected Calibration Error (ECE):**
$$
\text{ECE} = \sum_{m=1}^{M} \frac{n_m}{N} \left| \text{acc}(B_m) - \text{conf}(B_m) \right|
$$
where $B_m$ is the $m$-th bin, $\text{acc}(B_m)$ is the accuracy within that bin, and $\text{conf}(B_m)$ is the average predicted confidence.

In [ ]:
from sklearn.calibration import calibration_curve
import numpy as np

# Compute calibration curve
prob_true, prob_pred = calibration_curve(y_val, y_val_pred_proba, n_bins=10, strategy='uniform')

# Plot calibration curve
plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, marker='o', linewidth=2, label='Logistic Regression')
plt.plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Curve (Reliability Diagram)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Calculate Expected Calibration Error (ECE)
def expected_calibration_error(y_true, y_pred_proba, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_pred_proba, n_bins=n_bins, strategy='uniform')
    bin_counts = np.histogram(y_pred_proba, bins=n_bins, range=(0, 1))[0]
    bin_counts = bin_counts[bin_counts > 0]  # Only consider non-empty bins

    # Align prob_true and prob_pred with non-empty bins
    if len(prob_true) != len(bin_counts):
        # Recalculate to ensure alignment
        bins = np.linspace(0, 1, n_bins + 1)
        bin_indices = np.digitize(y_pred_proba, bins) - 1
        bin_indices = np.clip(bin_indices, 0, n_bins - 1)

        ece = 0.0
        for i in range(n_bins):
            mask = bin_indices == i
            if mask.sum() > 0:
                bin_acc = y_true[mask].mean()
                bin_conf = y_pred_proba[mask].mean()
                ece += (mask.sum() / len(y_true)) * np.abs(bin_acc - bin_conf)
        return ece
    else:
        ece = np.sum(bin_counts / len(y_true) * np.abs(prob_true - prob_pred))
        return ece

ece = expected_calibration_error(y_val, y_val_pred_proba, n_bins=10)
print(f"Expected Calibration Error (ECE): {ece:.4f}")

In [ ]:

# Evaluate the model
accuracy = sklearn.metrics.accuracy_score(y_val, y_val_pred)
roc_auc = sklearn.metrics.roc_auc_score(y_val, y_val_pred_proba)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation ROC AUC: {roc_auc:.4f}")


# Hyperparameter Tuning with Cross-Validation

![Cross-Validation Illustration](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)


- Goal: select model settings that generalize beyond the training set, not just fit it better.
- Use cross-validation on the training data only; never peek at the test set.
- Keep preprocessing inside a `Pipeline` to avoid leakage during CV.
- Choose a scoring function aligned to the objective (e.g., `roc_auc` or `log_loss` for probabilistic classifiers; `accuracy` when classes are balanced).
- Search strategies:
  - `GridSearchCV`: exhaustive over small, well-chosen grids.
  - `RandomizedSearchCV`: sample large spaces efficiently; good early-stage.
- Practical tips: set `refit=True` to retrain the best config on the full training split; inspect `cv_results_` for variance across folds; fix `random_state` for reproducibility; constrain search space to your compute budget.

Put together the training and validation sets because we are now using a dynamic train-validation split.

In [ ]:
print(f"{X_train_scaled.shape=:}, {y_train.shape=:}, {X_val_scaled.shape=:}, {y_val.shape=:}")
X_train_val_scaled = np.concatenate([X_train_scaled, X_val_scaled], axis=0)
y_train_val = np.concatenate([y_train, y_val], axis=0)
print(f"{X_train_val_scaled.shape=:}, {y_train_val.shape=:}")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Define the model.
model = sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)

# Define hyperparameter grid, we will search over these parameters over a specific range.
param_grid = {
    "C": [0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 500.0],
    # "penalty": ["l1", "l2"],
    # "solver": ["lbfgs", "liblinear"]
}

---
### Pseudocode: Grid Search with K-fold Cross-Validation

```python
inputs:
  base_model
  param_grid              # dict of hyperparameter lists
  X_train, y_train
  K                       # number of CV folds
  scoring_fn              # e.g., roc_auc, accuracy, neg_rmse

best_score := -inf
best_params := None

for params in cartesian_product(param_grid):
  fold_scores := []
  for (train_idx, val_idx) in KFold(n_splits=K, shuffle=True, random_state=42):
    X_tr := X_train[train_idx]; y_tr := y_train[train_idx]
    X_val := X_train[val_idx];  y_val := y_train[val_idx]

    # Build full pipeline to avoid leakage
    model := clone(base_model).set_params(**params)

    model.fit(X_tr, y_tr)

    # Use probabilities if the metric expects them; else labels
    y_pred := if scoring_fn expects probabilities then model.predict_proba(X_val)[:, 1]
              else model.predict(X_val)

    score := scoring_fn(y_val, y_pred)
    append(fold_scores, score)

  mean_score := mean(fold_scores)
  if mean_score > best_score:
    best_score := mean_score
    best_params := params

# Refit best configuration on all training data
best_model := Pipeline([("scaler", StandardScaler()),
                        ("model", clone(base_model).set_params(**best_params))])
best_model.fit(X_train, y_train)

return best_model, best_params, best_score
```

Notes:
- Keep preprocessing inside the CV pipeline; never touch the test set during tuning.
- Use `refit=True` to automatically retrain the best config on the full training split.
- Inspect `cv_results_` to understand variance across folds and confidence in the selected params.
---

In [ ]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    model,  # the model we want to tune.
    param_grid=param_grid,  # the parameters we want to search over.
    cv=5,  # the number of folds in cross-validation (5-fold CV in the visualization above.)
    scoring="roc_auc",  # the scoring function, what model is a good model?
    n_jobs=-1,  # the number of CPU cores to use, set to -1 to use all cores in parallel.
    verbose=1,  # the verbosity level.
)

In [ ]:
# Fit the grid search on scaled training data
print("Starting hyperparameter tuning...")
grid_search.fit(X_train_scaled, y_train)

In [ ]:
# Visualize grid search results
results_df = pd.DataFrame(grid_search.cv_results_)

# Extract C values for plotting
results_df['param_C'] = results_df['param_C']

# Plot: Line plot showing performance across C values
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.plot(results_df['param_C'], results_df['mean_test_score'],
        marker='o', linewidth=2, markersize=8, color='steelblue')

ax.set_xscale('log')
ax.set_xlabel('Regularization Parameter (C)', fontsize=12)
ax.set_ylabel('Mean ROC AUC (CV)', fontsize=12)
ax.set_title('Hyperparameter Tuning: C vs ROC AUC', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Alternative Method: RandomizedSearchCV
This is particularly useful when the parameter space is large and you only have a limited compute budget.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

# Define a broader parameter distribution for RandomizedSearchCV
param_distributions = {
    'C': loguniform(1e-3, 1e3),  # Sample C from a log-uniform distribution
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=42),
    param_distributions=param_distributions,
    n_iter=20,  # Number of parameter settings sampled
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Fit the random search
print("Starting randomized hyperparameter search...")
random_search.fit(X_train_scaled, y_train)

# Display results
print(f"\nBest parameters from RandomizedSearchCV: {random_search.best_params_}")
print(f"Best cross-validation ROC AUC: {random_search.best_score_:.4f}")


# Finally, examine the best model and its performance on the test set with `classification_report`

The `classification_report` function provides a comprehensive summary of classification performance metrics including precision, recall, F1-score, and support for each class. It's particularly useful for understanding model performance across different classes and identifying potential issues like class imbalance or poor performance on specific categories.


In [ ]:
from IPython.display import HTML
HTML('<iframe src="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html" width="100%" height="500"></iframe>')

In [ ]:
# Import classification_report if not already imported
from sklearn.metrics import classification_report

# Extract best model and parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best hyperparameters:", best_params)
print(f"Best cross-validation ROC AUC: {best_score:.4f}")

# Evaluate on test set
y_pred_test = best_model.predict(X_test_scaled)
y_proba_test = best_model.predict_proba(X_test_scaled)[:, 1]

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_roc_auc = roc_auc_score(y_test, y_proba_test)

print(f"\nTest Set Performance:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"ROC AUC: {test_roc_auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:")
print(cm)


# A Regression Example on the California Housing Dataset

In [ ]:
# Import necessary libraries for regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np

# Load California housing dataset
housing = fetch_california_housing()
X_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
y_housing = housing.target

print("California Housing Dataset")
print(f"Number of samples: {X_housing.shape[0]}")
print(f"Number of features: {X_housing.shape[1]}")
print(f"\nFeatures: {list(X_housing.columns)}")
print(f"\nTarget: Median house value (in $100,000s)")

# Display first few rows
print("\nFirst few rows:")
print(X_housing.head())
print("\nTarget statistics:")
print(f"Mean: ${y_housing.mean():.2f} (x100k)")
print(f"Std: ${y_housing.std():.2f} (x100k)")
print(f"Min: ${y_housing.min():.2f} (x100k)")
print(f"Max: ${y_housing.max():.2f} (x100k)")


In [ ]:
# Split data into train/validation/test
X_train, X_test, y_train, y_test = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=RANDOM_STATE
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=RANDOM_STATE
)  # 0.25 * 0.8 = 0.2 validation
print("Shapes:", X_train.shape, X_val.shape, X_test.shape)


In [ ]:
# Baselines: LinearRegression and untuned RandomForestRegressor
from sklearn.linear_model import LinearRegression

pipe_lin = sklearn.pipeline.Pipeline([
    ("scaler", sklearn.preprocessing.StandardScaler()),
    ("model", sklearn.linear_model.LinearRegression()),
])
pipe_lin.fit(X_train, y_train)

rf_base = RandomForestRegressor(random_state=RANDOM_STATE)
rf_base.fit(X_train, y_train)

for name, model in {"Linear": pipe_lin, "RF_base": rf_base}.items():
    ypred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, ypred))
    mae = mean_absolute_error(y_val, ypred)
    r2 = r2_score(y_val, ypred)
    print(f"{name} | RMSE={rmse:.3f} | MAE={mae:.3f} | R2={r2:.3f}")


In [ ]:
# Randomized search on RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(random_state=RANDOM_STATE)
param_dist_rf = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [None, 10, 20, 30, 40],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_gs = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist_rf,
    n_iter=1,  # adjust the number of parameter settings to sample depending on the compute budget.
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1,
    verbose=1,
    refit=True,
    random_state=RANDOM_STATE,
)

print("Starting RF randomized search ...")
rf_gs.fit(X_train, y_train)
print("Best params:", rf_gs.best_params_)
print("Best CV RMSE:", -rf_gs.best_score_)


In [ ]:
# Evaluate tuned RF on validation and test sets
best_rf_model = rf_gs.best_estimator_

# Validation
y_val_pred_best = best_rf_model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred_best))
mae_val = mean_absolute_error(y_val, y_val_pred_best)
r2_val = r2_score(y_val, y_val_pred_best)
print(f"Validation | RMSE={rmse_val:.3f} | MAE={mae_val:.3f} | R2={r2_val:.3f}")

# Test
y_test_pred_best = best_rf_model.predict(X_test)
rmse_te = np.sqrt(mean_squared_error(y_test, y_test_pred_best))
mae_te = mean_absolute_error(y_test, y_test_pred_best)
r2_te = r2_score(y_test, y_test_pred_best)
print(f"Test | RMSE={rmse_te:.3f} | MAE={mae_te:.3f} | R2={r2_te:.3f}")


In [ ]:
# Feature importance plot
importances = best_rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
features = np.array(X_housing.columns)[indices]
vals = importances[indices]

plt.figure(figsize=(8, 6))
plt.barh(features[::-1], vals[::-1])
plt.xlabel("Importance")
plt.title("RandomForestRegressor Feature Importances")
plt.tight_layout()
plt.show()


# Don't forget the attandance code!

# <img src="images/CME193_Lecture_6_Attendance_Form.png">